In [ ]:
import numpy as np
import pandas as pd
import math


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
data = pd.read_pickle('/content/drive/MyDrive/mreze/data/data/full_sequences.pkl')

In [ ]:
data.head()

,filename,mfccs,event_label,combine_squence_per_filename_and_event
0,2282.wav,"[[-659.70654, -618.7009, -581.4676, -577.1263,...",Dog,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
0,2282.wav,"[[-659.70654, -618.7009, -581.4676, -577.1263,...",Speech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2719.wav,"[[-584.7399, -515.89514, -473.47324, -464.3819...",Electric_shaver_toothbrush,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2719.wav,"[[-584.7399, -515.89514, -473.47324, -464.3819...",Speech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1238.wav,"[[-401.96198, -400.9539, -411.34525, -405.876,...",Blender,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
training = data[data['event_label'] == 'Alarm_bell_ringing']
X = np.array(training.mfccs.tolist())
y = np.array(training.combine_squence_per_filename_and_event.tolist())

X = np.einsum('ijk->ikj', X)

In [ ]:
def create_output_seq(y_orig):
  result_beginning = []
  result_end = []
  for sequence in y_orig:
    is_one = False
    new_sequence_beginning = []
    new_sequence_end = []
    to_assign_beginning = 0
    to_assign_end = 0
    for label in sequence:
      if label == 1:
        if is_one:
          to_assign_beginning = 0
        else:
          to_assign_beginning = 1
        is_one = True
      else:
        if is_one:
          new_sequence_end[len(new_sequence_end)-1] = 1
          to_assign_end = 0
        else:
          to_assign_end = 0
        is_one = False
      new_sequence_beginning.append(to_assign_beginning)
      new_sequence_end.append(to_assign_end)
    result_beginning.append(new_sequence_beginning)
    result_end.append(new_sequence_end)    
  return np.asarray(result_beginning), np.asarray(result_end)

In [ ]:
# Optimizovana sa paddingom (prosirenjem indikatora pocetka i kraja) - umesto jednog broja za pocetak i kraj -> (expansion_num * 2 +- 1) broj 
def create_output_seq_expanded(y_orig, expansion_num):
  result_beginning = []
  result_end = []
  for sequence in y_orig:
    new_sequence_beginning = [0] * len(sequence)
    new_sequence_end = [0] * len(sequence)
    indexes_beginning = [i for i in range(len(sequence)) if sequence[i] == 1 and i-1>=0 and i+1 < len(sequence) and sequence[i-1] == 0 and sequence[i+1] == 1] 
    indexes_end = [i for i in range(len(sequence)) if sequence[i] == 1 and i-1>=0 and i+1 < len(sequence) and sequence[i-1] == 1 and sequence[i+1] == 0] 

    for index in indexes_beginning:
      start = index - expansion_num if index - expansion_num >= 0 else index
      end = index + expansion_num if index + expansion_num < len(sequence) else index
      new_sequence_beginning[start:end] = [1] * (end - start)

    for index in indexes_end:
      start = index - expansion_num if index - expansion_num >= 0 else index
      end = index + expansion_num if index + expansion_num < len(sequence) else index
      new_sequence_end[start:end] = [1] * (end - start)

    result_beginning.append(new_sequence_beginning)
    result_end.append(new_sequence_end)    
  return np.asarray(result_beginning), np.asarray(result_end)

In [ ]:
y_new_beginning_exp, y_new_end_exp = create_output_seq_expanded(y, 2)
ind = 3
print(y[ind])
print(80*'-')
print(y_new_beginning_exp[ind])
print(80*'-')
print(y_new_end_exp[ind])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
--------------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
y_new_beginning, y_new_end = create_output_seq(y)

In [ ]:
y_new_beginning[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
y_new_end[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
y[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
X.shape

(1272, 431, 13)

In [ ]:
input_seq_len = 431
output_shape = 431
dense_layer_size = 150
num_of_features = 13
# output_len = len(data)

batch_size = 256  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 128  # Latent dimensionality of the encoding space.

In [ ]:
# Define architecture of autoencoder model. 
import tensorflow as tf
from tensorflow import keras

def create_encoder_decoder(input_seq_len = 431, num_of_features = 13):
  # Define an input sequence and process it.

  encoder_inputs = keras.Input(shape=(input_seq_len, num_of_features))
  encoder = keras.layers.GRU(latent_dim, return_sequences=True, return_state=True)
  encoder_outputs, state_h = encoder(encoder_inputs)

  encoder_states = [state_h]
  # repeat_vector = keras.layers.RepeatVector(input_seq_len)(encoder_outputs)
  # Set up the decoder, using `encoder_states` as initial state.
  # decoder_inputs = keras.Input(shape=(25, 773))

  # We set up our decoder to return full output sequences.
  # decoder_gru = keras.layers.GRU(latent_dim, return_sequences=True, return_state=True)
  # decoder_outputs, _ = decoder_gru(encoder_outputs, initial_state=encoder_states)
  # decoder_dense = keras.layers.Dense(1, activation="sigmoid")
  # decoder_outputs = decoder_dense(decoder_outputs)

  decoder_beg = keras.layers.Dense(dense_layer_size, activation='relu')
  decoder_beg_outputs_0 = decoder_beg(encoder_outputs)

  # decoder_beg_1 = keras.layers.Dense(dense_layer_size, activation='relu')
  # decoder_beg_outputs_1 = decoder_beg_1(decoder_beg_outputs_0)

  decoder_beg_2 = keras.layers.Dense(1, activation='sigmoid')
  decoder_beg_outputs_2 = decoder_beg_2(decoder_beg_outputs_0)

  decoder_end = keras.layers.Dense(dense_layer_size, activation='relu')
  decoder_end_outputs_0 = decoder_end(encoder_outputs)

  # decoder_end_1 = keras.layers.Dense(dense_layer_size, activation='relu')
  # decoder_end_outputs_1 = decoder_end_1(decoder_end_outputs_0)

  decoder_end_2 = keras.layers.Dense(1, activation='sigmoid')
  decoder_end_outputs_2 = decoder_end_2(decoder_end_outputs_0)

  return encoder_inputs, decoder_beg_outputs_2, decoder_end_outputs_2
  


In [ ]:
encoder_inputs, decoder_outputs_beg, decoder_outputs_end = create_encoder_decoder()
model = keras.Model(encoder_inputs, [decoder_outputs_beg, decoder_outputs_end])
model.summary()

Model: "model_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_32 (InputLayer)           [(None, 431, 13)]    0                                            
__________________________________________________________________________________________________
gru_30 (GRU)                    [(None, 431, 128), ( 54912       input_32[0][0]                   
__________________________________________________________________________________________________
dense_144 (Dense)               (None, 431, 150)     19350       gru_30[0][0]                     
__________________________________________________________________________________________________
dense_146 (Dense)               (None, 431, 150)     19350       gru_30[0][0]                     
___________________________________________________________________________________________

In [ ]:
y_new_beginning.shape

(392, 431)

In [ ]:
model.compile(
    optimizer="adam", loss=tf.keras.losses.MeanSquaredError(), metrics=["accuracy"]
)
model.fit(
    [X],
    [y_new_beginning_exp, y_new_end_exp],
    batch_size=batch_size,
    epochs=300,
    validation_split=0.2,
)

Epoch 1/300
2/2 [==============================] - 3s 742ms/step - loss: 0.0296 - dense_145_loss: 0.0151 - dense_147_loss: 0.0145 - dense_145_accuracy: 0.9833 - dense_147_accuracy: 0.9851 - val_loss: 0.0375 - val_dense_145_loss: 0.0195 - val_dense_147_loss: 0.0180 - val_dense_145_accuracy: 0.9794 - val_dense_147_accuracy: 0.9812
Epoch 2/300
2/2 [==============================] - 0s 157ms/step - loss: 0.0295 - dense_145_loss: 0.0151 - dense_147_loss: 0.0144 - dense_145_accuracy: 0.9833 - dense_147_accuracy: 0.9851 - val_loss: 0.0380 - val_dense_145_loss: 0.0201 - val_dense_147_loss: 0.0179 - val_dense_145_accuracy: 0.9791 - val_dense_147_accuracy: 0.9812
Epoch 3/300
2/2 [==============================] - 0s 145ms/step - loss: 0.0294 - dense_145_loss: 0.0150 - dense_147_loss: 0.0144 - dense_145_accuracy: 0.9834 - dense_147_accuracy: 0.9851 - val_loss: 0.0371 - val_dense_145_loss: 0.0192 - val_dense_147_loss: 0.0179 - val_dense_145_accuracy: 0.9794 - val_dense_147_accuracy: 0.9812
Epoch 4

In [ ]:
def combine_beginning_end(y_beg, y_end):
  y_final = []
  was_one = False
  to_append_new = 0
  to_append = 0
  for i in range(0, len(y_beg)):
    if y_beg[i] == 1:
      if was_one:
        y_final[index_last_beginning:i] = [0] * (i - index_last_beginning)
      else:
        to_append_new = 1
        was_one = True
      index_last_beginning = i  
      
    if y_end[i] == 1:
      if was_one:
        to_append_new = 0
        was_one = False
      # else:
      #   to_append = 1
      #   was_one = True
    y_final.append(to_append) 
    to_append = to_append_new 
  return y_final 

In [ ]:
num = 6
predicted = model.predict([np.array([X[num]])])

print(np.round(np.concatenate(predicted[0][0], axis=0), 3))
# print(np.concatenate(predicted[0][0], axis=0))
# print(np.round(predicted[0][0], 3))
# print(predicted[0][0])
print("-------------------------------- BEGINNINGS --------------------------------")
predicted_beg_test = np.round(np.concatenate(predicted[0][0], axis=0), 0).astype(int)
print(predicted_beg_test)
# print(y_new_beginning[num])
print("-------------------------------- ENDINGS --------------------------------")
predicted_end_test = np.round(np.concatenate(predicted[1][0], axis=0), 0).astype(int)
print(predicted_end_test)
# print(y_new_end[num])
print("-------------------------------- ORIGINAL --------------------------------")

print(y[num])

[0.    0.    0.    0.001 0.001 0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.74  0.986 0.929 0.128 0.004 0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.019 0.088 0.072 0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.001 0.003 0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.001 0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.002 0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.018 0.023 0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0

In [ ]:
print(predicted)

In [ ]:
print(np.array(combine_beginning_end(predicted_beg_test, predicted_end_test)))
print("-------------------------------- ORIGINAL --------------------------------")
print(y[num])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
-------------------------------- ORIGINAL --------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0